In [2]:
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import database as db
import gdal
import rasterio
from rasterstats import zonal_stats
from munging import get_centroid
from apsim.daymet import create_excel_met
from apsim.apsim_input_writer import create_mukey_runs
import matplotlib.pyplot as plt
#ogr2ogr -f "PostgreSQL" PG:"dbname=aepe port=5432 user=mjn password=KBlhsab7618 host=10.24.18.212" "accola_yield.geojson" -skip-failures

In [3]:
dbconn = db.connect_to_db('database.ini')

In [3]:
accola_soy_mgmt_2018 = json.loads( open( 'crop_jsons/accola_sfc_2018.json', 'r' ).read() )
accola_corn_mgmt_2019 = json.loads( open( 'crop_jsons/accola_cfs_2019.json', 'r' ).read() )

In [5]:
accola_soils = gpd.read_postgis('SELECT * FROM biocent_farms.accola_ssurgo;', dbconn, geom_col='wkb_geometry')
accola_mukeys = list(np.unique(accola_soils['mukey']))
#accola_soils = pd.read_sql('SELECT * FROM biocent_farms.accola_ssurgo;', dbconn)
#### Get centroid and create met file
#accola_centroid = get_centroid(accola_soils, 'areasymbol', 'wkb_geometry')
#create_excel_met(accola_centroid[0], accola_centroid[1], 2012, 2019, 'accola')

In [ ]:
create_mukey_runs(accola_mukeys, dbconn, 'sfc', 'accola.met', 'Accola', sfc_mgmt=accola_soy_mgmt_2018, cfs_mgmt=accola_corn_mgmt_2019)

In [ ]:
#gdal Rasterize (vector to raster)
#raster analysis > zonal statistics
#user python rasterstats to calculate zonal statistics
# from rasterstats import zonal_stats
# stats = zonal_stats("tests/data/polygons.shp", "tests/data/elevation.tif")
# stats[1].keys()
# ['count', 'min', 'max', 'mean']
# [f['mean'] for f in stats]
# [756.6057470703125, 114.660084635416666]

In [4]:
accola_yield = gpd.read_file("yield_monitor/Accola2019YieldwGeom.dbf")

In [5]:
accola_yield_clean = accola_yield[['Yld_Vol_Dr','geometry']]
#accola_yield_clean['Yld_Vol_Dr'].describe()

In [ ]:
# For re-projecting input vector layer to raster projection
def reproject(vector_gpd, raster):
    proj = raster.crs.to_proj4()
    print("Original vector layer projection: ", vector_gpd.crs)
    reproj = vector_gpd.to_crs(proj)
    print("New vector layer projection (PROJ4): ", reproj.crs)
    fig, ax = plt.subplots(figsize=(15, 15))
    rplt.show(raster, ax=ax)
    reproj.plot(ax=ax, facecolor='none', edgecolor='red')
    fig.show()
    return reproj

In [ ]:
#stats list: ['min', 'max', 'mean', 'count', 'sum', 'std', 'median', 'majority', 'minority', 'unique', 'range']
def get_zonal_stats(vector, raster, stats):
    # Run zonal statistics, store result in geopandas dataframe
    result = zonal_stats(vector, raster, stats=stats, geojson_out=True)
    geostats = gpd.GeoDataFrame.from_features(result)
    return geostats

In [ ]:
# For generating raster from zonal statistics result
def stats_to_raster(zdf, raster, stats, out_raster, no_data='y'):
    meta = raster.meta.copy()
    out_shape = raster.shape
    transform = raster.transform
    dtype = raster.dtypes[0]
    field_list = list_columns(stats)
    index = int(input("Rasterize by which field? "))
    zone = zdf[field_list[index]]
    shapes = ((geom,value) for geom, value in zip(zdf.geometry, zone))
    burned = rasterize(shapes=shapes, fill=0, out_shape=out_shape, transform=transform)
    show(burned)
    meta.update(dtype=rasterio.float32, nodata=0)
    # Optional to set nodata values to min of stat
    if no_data == 'y':
        cutoff = min(zone.values)
        print("Setting nodata cutoff to: ", cutoff)
        burned[burned < cutoff] = 0 
    with rasterio.open(out_raster, 'w', **meta) as out:
        out.write_band(1, burned)
    print("Zonal Statistics Raster generated")

In [8]:
stats = zonal_stats('yield_monitor/accola_soils.shp', 'yield_monitor/accola_2019_yield_raster.tif', geojson_out=True)

In [9]:
gpd.GeoDataFrame.from_features(stats)

,geometry,objectid,areasymbol,spatialver,musym,mukey,shape_leng,shape_area,_count,_sum,...,_sum_1,_mean_1,_median,_stdev,_min,_max,min,max,mean,count
0,"MULTIPOLYGON (((-93.67114 41.97944, -93.67177 ...",1670419,IA169,9.0,L107,2835021,4532.587964,17372.778909,1027.0,1.876259e+05,...,1.876259e+05,182.693177,186.720093,26.237834,66.781502,245.805603,66.781502,245.805603,182.693190,1027
1,"POLYGON ((-93.67638 41.98140, -93.67645 41.981...",1699584,IA169,9.0,L95,2835194,427.132962,9608.816987,617.0,1.001452e+05,...,1.001452e+05,162.309815,175.093201,55.368015,40.117100,273.226410,40.117100,273.226410,162.309816,617
2,"POLYGON ((-93.66838 41.98085, -93.66838 41.980...",1673500,IA169,9.0,L138B,2765537,5151.993709,51175.383764,3105.0,6.888473e+05,...,6.888473e+05,221.850985,223.410599,31.126392,66.104897,348.802002,66.104897,348.802002,221.850986,3105
3,"POLYGON ((-93.67787 41.97963, -93.67787 41.979...",1674524,IA169,9.0,6,411333,370.991947,422.984319,39.0,5.831197e+03,...,5.831197e+03,149.517880,157.001404,20.448641,102.758202,179.421295,102.758202,179.421295,149.517879,39
4,"POLYGON ((-93.66838 41.97973, -93.66838 41.979...",1675300,IA169,9.0,L107,2835021,3939.586786,3689.868249,226.0,3.998885e+04,...,3.998885e+04,176.941802,187.816048,48.116209,42.994301,337.525299,42.994301,337.525299,176.941787,226
5,"POLYGON ((-93.67577 41.98026, -93.67571 41.980...",1679582,IA169,9.0,L138B,2765537,550.387774,11041.134105,642.0,1.279321e+05,...,1.279321e+05,199.271186,201.629395,30.809508,82.115501,282.751801,82.115501,282.751801,199.271174,642
6,"POLYGON ((-93.67305 41.98305, -93.67261 41.983...",1679975,IA169,9.0,L138B,2765537,1300.434886,5963.292117,365.0,7.641316e+04,...,7.641316e+04,209.351127,210.781097,32.295120,69.628998,336.448303,69.628998,336.448303,209.351113,365
7,"POLYGON ((-93.67784 41.97951, -93.67786 41.979...",1689022,IA169,9.0,L507,2922031,42460.177496,131704.052460,7975.0,1.570600e+06,...,1.570600e+06,196.940452,203.940308,44.503792,40.268799,346.724609,40.268799,346.724609,196.940470,7975
8,"MULTIPOLYGON (((-93.66842 41.98255, -93.66840 ...",1689289,IA169,9.0,L55,2800480,922.446488,6182.684343,663.0,1.145793e+05,...,1.145793e+05,172.819454,182.495895,42.923843,40.689999,311.745087,40.689999,311.745087,172.819452,663
9,"POLYGON ((-93.67227 41.97944, -93.67241 41.979...",1689439,IA169,9.0,L55,2800480,524.713926,6250.414950,365.0,5.170339e+04,...,5.170339e+04,141.653134,141.478699,43.498758,53.092400,226.033203,53.092400,226.033203,141.653136,365


In [ ]:
#gdal.Grid('yield_avg.tif', 'accola_yield_basic.csv')

In [ ]:
len(accola_yield_clean.loc[accola_yield_clean['Yld_Vol_Dr'] <= 150])

In [ ]:
accola_mukeys = list(np.unique(accola_soils['mukey']))

In [ ]:
create_mukey_runs(accola_mukeys, dbconn, 'sfc', 'accola.met', 'Accola', sfc_mgmt=accola_soy_mgmt_2018, cfs_mgmt=accola_corn_mgmt_2019)